# 컨텐츠 기반 추천시스템 만들기- TMDB 5000 Movie Dataset 
*TMDB5000Movie 데이터셋을 가지고 영화 추천 시스템을 만들자!
* 함께 비교하면 좋을 상품 : 코사인 유사도를 통해서 구현할 수 있어!

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import json

In [80]:
data = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/tmdb_5000_movies.csv')

In [81]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [6]:
from ydata_profiling import ProfileReport
profile = ProfileReport(data, title="Profile Report")
profile.to_file("movie_set.html")

C:\Users\hi\anaconda3\envs\testenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Export report to file: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 47.37it/s]


In [7]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [55]:
data.isna().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [66]:
data['overview'] = data['overview'].dropna()

In [68]:
data.isna().sum() # drop na 했으면 없어야하는 거 아냐??

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [83]:
type(data['genres'][0]) # 얘는 Json파일이겠구나

str

In [84]:
data['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [87]:
test = json.loads(data['genres'][0]) 
test

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

이렇게 해주면 data['genres'][0]는 JSON이 되는거야. 리스트 안에 있으니까 반복문에 넣어서 뽑아야겠네.

In [ ]:
for i in test:
    print(list(i.values())[1])

첫번째 방법

In [92]:
for i in test:
    print(i['name'])

Action
Adventure
Fantasy
Science Fiction


이렇게 두 가지 방법이 있어.

In [93]:
genre = []
for i in test:
    genre.append(i['name'])

In [94]:
genre # 리스트로 나오도록

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [95]:
genre = " ".join(genre)
genre

'Action Adventure Fantasy Science Fiction'

In [97]:
data['genres'] = data['genres'].apply(json.loads)

In [101]:
data['genres'] = data['genres'].apply(lambda x : [i['name'] for i in x])

In [103]:
data['genres'] = data['genres'].apply(lambda x : " ".join(x))

In [104]:
data['genres']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres, Length: 4803, dtype: object

In [106]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [107]:
data.columns # 같이 name에 들어가있으니까 반복문에 넣어주면 됨.

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [ ]:
['keywords', 'production_companies', 'production_countries','spoken_languages']

In [108]:
# 똑같이 딕셔너리에 있는 것들을 json으로 바꾸고 뽑아오는 것을 for문에 넣어서 만들기.
for col in ['keywords', 'production_companies', 'production_countries','spoken_languages']:
    data[col] = data[col].apply(json.loads)
    data[col] = data[col].apply(lambda x : [i['name'] for i in x])
    data[col] = data[col].apply(lambda x : " ".join(x))

In [109]:
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners Twentieth Century Fox ...,United States of America United Kingdom,2009-12-10,2787965087,162.0,English Español,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures Jerry Bruckheimer Films S...,United States of America,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia Pictures Danjaq B24,United Kingdom United States of America,2015-10-26,880674609,148.0,Français English Español Italiano Deutsch,Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary Pictures Warner Bros. DC Entertainme...,United States of America,2012-07-16,1084939099,165.0,English,Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,United States of America,2012-03-07,284139100,132.0,English,Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [110]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [112]:
# 컬럼이 너무 많을 때 몇몇 컬럼만 뽑아서 쓸 경우 이렇게 한다.
data = data[['id', 'title', 'genres', 'vote_average', 'popularity', 'keywords', 'overview']]
data.head()

,id,title,genres,vote_average,popularity,keywords,overview
0,19995,Avatar,Action Adventure Fantasy Science Fiction,7.2,150.437577,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,6.9,139.082615,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,Action Adventure Crime,6.3,107.376788,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller,7.6,112.312950,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...
4,49529,John Carter,Action Adventure Science Fiction,6.1,43.926995,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca..."


* 장르의 단어를 벡터화해서 코사인유사도를 통해 비슷한 장르를 가진 행을 찾고, 
* 유사도가 높은 행을 기준으로 내림차순 정렬

In [114]:
from sklearn.feature_extraction.text import CountVectorizer

In [115]:
cv = CountVectorizer(ngram_range=(1,2))
genre_mat = cv.fit_transform(data['genres'])
print(genre_mat)

  (0, 0)	1
  (0, 16)	1
  (0, 124)	1
  (0, 232)	1
  (0, 138)	1
  (0, 1)	1
  (0, 24)	1
  (0, 135)	1
  (0, 233)	1
  (1, 0)	1
  (1, 16)	1
  (1, 124)	1
  (1, 24)	1
  (1, 125)	1
  (2, 0)	1
  (2, 16)	1
  (2, 1)	1
  (2, 64)	1
  (2, 20)	1
  (3, 0)	1
  (3, 64)	1
  (3, 90)	1
  (3, 234)	1
  (3, 4)	1
  (3, 68)	1
  :	:
  (4796, 90)	1
  (4796, 234)	1
  (4796, 106)	1
  (4796, 144)	1
  (4797, 234)	1
  (4797, 153)	1
  (4797, 157)	1
  (4798, 0)	1
  (4798, 64)	1
  (4798, 234)	1
  (4798, 4)	1
  (4798, 78)	1
  (4799, 44)	1
  (4799, 214)	1
  (4799, 58)	1
  (4800, 90)	1
  (4800, 44)	1
  (4800, 214)	1
  (4800, 104)	1
  (4800, 50)	1
  (4800, 250)	1
  (4800, 182)	1
  (4800, 251)	1
  (4800, 229)	1
  (4802, 80)	1


* ngram_range=(1,2) 글자를 한 글자씩 하겠다.

In [116]:
genre_mat.shape #4803행 276열이 있는 벡터로 만들어버림.

(4803, 276)

* 코사인 유사도를 사용해 genre_mat의 유사도 산출 후 genre_sim에 저장
* 비슷한 장르 영화를 찾기 위해서 계산
* 코사인 유사도는 주로 문자, 문서의 유사도 계산에 사용됨.

In [117]:
from sklearn.metrics.pairwise import cosine_similarity

In [121]:
genre_sim = cosine_similarity(genre_mat, genre_mat) # 행에 genre_mat 넣고 열에 genre_mat을 넣어서 상관분석 처럼 코사인유사도 측정.
print(genre_sim.shape)
print(genre_sim[:10])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 ...
 [0.75592895 0.3380617  0.50709255 ... 0.         0.         0.        ]
 [0.4472136  0.6        0.2        ... 0.         0.         0.        ]
 [0.74535599 0.8        0.6        ... 0.         0.         0.        ]]


In [120]:
data['genres'][:5]

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
2                      Action Adventure Crime
3                 Action Crime Drama Thriller
4            Action Adventure Science Fiction
Name: genres, dtype: object

In [131]:
a = np.array([34,1,47,466666,5])

In [132]:
a.argsort() # 숫자 순서대로 인덱스가 나오는 것.

array([1, 4, 0, 2, 3], dtype=int64)

In [134]:
a[::-1]

array([     5, 466666,     47,      1,     34])

In [135]:
[34,1,47,466666,5][::-1] #역순정렬

[5, 466666, 47, 1, 34]

In [137]:
sorted_genre_sim = genre_sim.argsort()[:, ::-1]

In [138]:
sorted_genre_sim[:1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401]], dtype=int64)

In [139]:
data.loc[0]

id                                                          19995
title                                                      Avatar
genres                   Action Adventure Fantasy Science Fiction
vote_average                                                  7.2
popularity                                             150.437577
keywords        culture clash future space war space colony so...
overview        In the 22nd century, a paraplegic Marine is di...
Name: 0, dtype: object

In [140]:
data.loc[3494]

id                                                          27549
title                   Beastmaster 2: Through the Portal of Time
genres                   Action Adventure Fantasy Science Fiction
vote_average                                                  4.6
popularity                                               1.478505
keywords        based on novel time travel sequel psychotronic...
overview        Mark Singer returns as Dar, the warrior who ca...
Name: 3494, dtype: object

In [141]:
data.loc[813] # 내가 좋아하는 영화와 장르가 비슷한 애들

id                                                           1924
title                                                    Superman
genres                   Action Adventure Fantasy Science Fiction
vote_average                                                  6.9
popularity                                              48.507081
keywords        saving the world journalist dc comics crime fi...
overview        Mild-mannered Clark Kent works as a reporter a...
Name: 813, dtype: object

In [145]:
data[data['title'] ==  'Superman']

,id,title,genres,vote_average,popularity,keywords,overview
813,1924,Superman,Action Adventure Fantasy Science Fiction,6.9,48.507081,saving the world journalist dc comics crime fi...,Mild-mannered Clark Kent works as a reporter a...


In [146]:
data[data['title'] ==  'Superman'].index.values


array([813], dtype=int64)

In [153]:
data.iloc[sorted_genre_sim[813,: (5)].reshape(-1)]

,id,title,genres,vote_average,popularity,keywords,overview
0,19995,Avatar,Action Adventure Fantasy Science Fiction,7.2,150.437577,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di..."
3494,27549,Beastmaster 2: Through the Portal of Time,Action Adventure Fantasy Science Fiction,4.6,1.478505,based on novel time travel sequel psychotronic...,"Mark Singer returns as Dar, the warrior who ca..."
813,1924,Superman,Action Adventure Fantasy Science Fiction,6.9,48.507081,saving the world journalist dc comics crime fi...,Mild-mannered Clark Kent works as a reporter a...
870,8536,Superman II,Action Adventure Fantasy Science Fiction,6.5,30.515175,saving the world dc comics sequel superhero ba...,Three escaped criminals from the planet Krypto...
46,127585,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,7.5,118.078691,1970s mutant time travel marvel comic based on...,The ultimate X-Men ensemble fights a war for t...


In [154]:
def find_sim_movie(df, sorted_genre_sim, title_name, top_n=10): # top_n=10  기본값을 10으로 받겠다.
    title_movie = df[df['title'] == title_name]
    title_index =  title_movie.index.values
    similar_indexes = sorted_genre_sim[title_index, : (top_n)]
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [155]:
find_sim_movie(data, sorted_genre_sim, 'Avatar', 10) # 10개 추천해줘

[[   0 3494  813  870   46   14 1296 1652  419  420]]


,id,title,genres,vote_average,popularity,keywords,overview
0,19995,Avatar,Action Adventure Fantasy Science Fiction,7.2,150.437577,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di..."
3494,27549,Beastmaster 2: Through the Portal of Time,Action Adventure Fantasy Science Fiction,4.6,1.478505,based on novel time travel sequel psychotronic...,"Mark Singer returns as Dar, the warrior who ca..."
813,1924,Superman,Action Adventure Fantasy Science Fiction,6.9,48.507081,saving the world journalist dc comics crime fi...,Mild-mannered Clark Kent works as a reporter a...
870,8536,Superman II,Action Adventure Fantasy Science Fiction,6.5,30.515175,saving the world dc comics sequel superhero ba...,Three escaped criminals from the planet Krypto...
46,127585,X-Men: Days of Future Past,Action Adventure Fantasy Science Fiction,7.5,118.078691,1970s mutant time travel marvel comic based on...,The ultimate X-Men ensemble fights a war for t...
14,49521,Man of Steel,Action Adventure Fantasy Science Fiction,6.5,99.398009,saving the world dc comics superhero based on ...,A young boy learns that he has extraordinary p...
1296,9531,Superman III,Comedy Action Adventure Fantasy Science Fiction,5.3,22.164202,saving the world dc comics super computer iden...,"Aiming to defeat the Man of Steel, wealthy exe..."
1652,14164,Dragonball Evolution,Action Adventure Fantasy Science Fiction Thriller,2.9,21.677732,karate superhero revenge dragon duringcreditss...,The young warrior Son Goku sets out on a quest...
419,8247,Jumper,Adventure Fantasy Science Fiction,5.9,21.218000,adolescence based on novel loss of child fight...,"David Rice is a man who knows no boundaries, a..."
420,11253,Hellboy II: The Golden Army,Adventure Fantasy Science Fiction,6.5,58.579760,auction northern ireland resignation superhero...,In this continuation to the adventure of the d...
